In [1]:
# Step 1: Set up the environment in Google Colab
!pip install transformers
!pip install datasets
!pip install py7zr

# Step 2: Load the dataset and preprocess
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the dataset
dataset = load_dataset("samsum")
train_data = dataset["train"]

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

def preprocess_function(examples):
    inputs = examples["dialogue"]
    targets = examples["summary"]
    inputs = [doc + tokenizer.eos_token for doc in inputs]
    model_inputs = tokenizer(inputs, max_length=512, padding=True, truncation=True, return_tensors="pt")

    # Setup the target for the loss (classification)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=150, padding=True, truncation=True, return_tensors="pt")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Preprocess the data
train_data = train_data.map(preprocess_function, batched=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [10]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForSeq2SeqLM, AdamW
from torch.nn.utils.rnn import pad_sequence

# Load the pre-trained model
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Create a custom collate function to pad the sequences within the batch
def collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]
    labels = [item["labels"] for item in batch]

    input_ids = pad_sequence([torch.tensor(ids) for ids in input_ids], batch_first=True)
    attention_mask = pad_sequence([torch.tensor(mask) for mask in attention_mask], batch_first=True)
    labels = pad_sequence([torch.tensor(ids) for ids in labels], batch_first=True)

    return {
        "input_ids": input_ids.to(device),
        "attention_mask": attention_mask.to(device),
        "labels": labels.to(device)
    }

train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=collate_fn)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Save the fine-tuned model and tokenizer to Google Drive
output_dir = "/content/drive/My Drive/NLP_Project/FineTunedModel/"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Loss: 0.4791097640991211
Epoch 2/10, Loss: 0.4270494282245636
Epoch 3/10, Loss: 0.8256051540374756
Epoch 4/10, Loss: 0.7209801077842712
Epoch 5/10, Loss: 0.2849361002445221
Epoch 6/10, Loss: 0.5696766972541809
Epoch 7/10, Loss: 0.16968096792697906
Epoch 8/10, Loss: 0.6650962233543396
Epoch 9/10, Loss: 0.49338194727897644
Epoch 10/10, Loss: 0.5467504262924194


('/content/drive/My Drive/NLP_Project/FineTunedModel/tokenizer_config.json',
 '/content/drive/My Drive/NLP_Project/FineTunedModel/special_tokens_map.json',
 '/content/drive/My Drive/NLP_Project/FineTunedModel/tokenizer.json')

In [4]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a0bdf90514de7a3e0d36d9ead3459a8ccb7247ff83870f2eb8b70407a8c504ab
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [6]:
print(test_data[0].keys())


dict_keys(['id', 'dialogue', 'summary'])


In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset
from rouge_score import rouge_scorer

# Load the pre-trained model and tokenizer
model_name = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the test dataset
dataset = load_dataset("samsum")
test_data = dataset["test"]

# Define a function to generate summaries using the fine-tuned model
def generate_summary(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

# Calculate ROUGE scores
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
references = [data["summary"] for data in test_data]
generated_summaries = [generate_summary(data["dialogue"]) for data in test_data]

# Calculate ROUGE scores for each generated summary
scores = []
for summary, reference in zip(generated_summaries, references):
    score = rouge_scorer.score(summary, reference)
    scores.append(score)

print("ROUGE Scores:")
for idx, score in enumerate(scores, 1):
    print(f"Summary {idx}: {score}")


ROUGE Scores:
Summary 1: {'rouge1': Score(precision=0.0625, recall=0.14285714285714285, fmeasure=0.08695652173913043), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0625, recall=0.14285714285714285, fmeasure=0.08695652173913043)}
Summary 2: {'rouge1': Score(precision=0.08333333333333333, recall=0.1111111111111111, fmeasure=0.09523809523809525), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.08333333333333333, recall=0.1111111111111111, fmeasure=0.09523809523809525)}
Summary 3: {'rouge1': Score(precision=0.13793103448275862, recall=0.4, fmeasure=0.20512820512820515), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.13793103448275862, recall=0.4, fmeasure=0.20512820512820515)}
Summary 4: {'rouge1': Score(precision=0.09090909090909091, recall=0.125, fmeasure=0.10526315789473685), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0909090909